In [16]:
import cv2
import urllib
import numpy             as np
import pandas            as pd
import seaborn           as sns
import random
import os
import glob
import shutil
from imutils        import paths
import matplotlib.pyplot as plt
from sklearn.utils  import shuffle
from urllib.request import urlopen
from sklearn.decomposition import PCA
import xml.etree.ElementTree as ET
from collections import Counter

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models              import Sequential
from tensorflow.keras.preprocessing       import image
from tensorflow.keras.utils               import to_categorical
from tensorflow.keras.callbacks           import ModelCheckpoint,EarlyStopping
from tensorflow.keras.layers              import Conv2D,Flatten,MaxPooling2D,Dense,Dropout,SpatialDropout2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img,array_to_img

In [17]:
import os
import json

# Veri seti yolu
dataset_path = "Pascal_VOC_2012_Dataset"
train_annotations_path = os.path.join(dataset_path, "train", "_annotations.coco.json")

# COCO formatındaki JSON'u oku
with open(train_annotations_path, "r") as f:
    data = json.load(f)

# ID'den kategori ismi eşlemesi oluştur
categories = {cat["id"]: cat["name"] for cat in data["categories"]}

# İlk 10 görsel ve etiketleri yazdır
for image_info in data["images"][:10]:
    image_id = image_info["id"]
    image_path = os.path.join(dataset_path, "train", image_info["file_name"])

    # Bu görsele ait annotation'ları bul
    annotations = [ann for ann in data["annotations"] if ann["image_id"] == image_id]

    objects = []
    for ann in annotations:
        class_name = categories[ann["category_id"]]
        bbox = ann["bbox"]  # COCO formatı [x, y, width, height]
        xmin, ymin, width, height = bbox
        xmax = xmin + width
        ymax = ymin + height

        objects.append({
            "class_name": class_name,
            "bbox": [xmin, ymin, xmax, ymax]
        })

    print(f"Görsel: {image_path}")
    print("Nesneler:", objects)


Görsel: Pascal_VOC_2012_Dataset\train\2012_004107_jpg.rf.514479ee455502d126b4540aada33375.jpg
Nesneler: [{'class_name': 'person', 'bbox': [67, 57, 367, 359]}]
Görsel: Pascal_VOC_2012_Dataset\train\2010_001699_jpg.rf.519e087484ecf63279b7fc8020b61f0b.jpg
Nesneler: [{'class_name': 'person', 'bbox': [269, 90, 419, 374]}, {'class_name': 'chair', 'bbox': [257, 83, 420, 357]}, {'class_name': 'pottedplant', 'bbox': [455, 99, 499, 168]}, {'class_name': 'pottedplant', 'bbox': [198, 17, 409, 157]}]
Görsel: Pascal_VOC_2012_Dataset\train\2010_002044_jpg.rf.51259121f651a2a5c18927fddc306371.jpg
Nesneler: [{'class_name': 'car', 'bbox': [37, 29, 471, 327]}, {'class_name': 'car', 'bbox': [398, 77, 499, 242]}, {'class_name': 'person', 'bbox': [62, 9, 92, 120]}, {'class_name': 'person', 'bbox': [197, 12, 221, 42]}, {'class_name': 'person', 'bbox': [191, 94, 214, 178]}, {'class_name': 'person', 'bbox': [220, 79, 290, 163]}, {'class_name': 'person', 'bbox': [350, 68, 438, 132]}, {'class_name': 'person', 'bb

In [18]:
import os
import tensorflow as tf
import xml.etree.ElementTree as ET
import numpy as np

# Veri seti dizini
dataset_path = "pascal-voc-2012"
image_dir = os.path.join(dataset_path, "JPEGImages")
annotation_dir = os.path.join(dataset_path, "Annotations")

# Sınıf etiketleri
class_names = ["aeroplane", "bicycle", "bird", "boat", "bottle",
               "bus", "car", "cat", "chair", "cow", "diningtable",
               "dog", "horse", "motorbike", "person", "pottedplant",
               "sheep", "sofa", "train", "tvmonitor"]

# XML'i TensorFlow için uygun hale getirme
def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    filename = root.find("filename").text
    image_path = os.path.join(image_dir, filename)

    objects = []
    for obj in root.findall("object"):
        class_name = obj.find("name").text
        bbox = obj.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        ymin = int(bbox.find("ymin").text)
        xmax = int(bbox.find("xmax").text)
        ymax = int(bbox.find("ymax").text)

        objects.append({
            "class_name": class_name,
            "bbox": [xmin, ymin, xmax, ymax]
        })

    return image_path, objects

# İlk 10 veriyi gösterelim
for xml_file in os.listdir(annotation_dir)[:10]:
    xml_path = os.path.join(annotation_dir, xml_file)
    image_path, objects = parse_xml(xml_path)
    print(f"Görsel: {image_path}")
    print("Nesneler:", objects)


FileNotFoundError: [WinError 3] Sistem belirtilen yolu bulamıyor: 'pascal-voc-2012\\Annotations'

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import cv2

# TensorFlow veri kümesi oluşturma fonksiyonu
def load_data():
    images, bboxes, labels = [], [], []

    for xml_file in os.listdir(annotation_dir):
        xml_path = os.path.join(annotation_dir, xml_file)
        image_path, objects = parse_xml(xml_path)

        # Görseli yükle ve normalize et
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224)) / 255.0  # Normalize et

        images.append(image)
        boxes = []
        classes = []

        for obj in objects:
            bbox = obj["bbox"]
            class_index = class_names.index(obj["class_name"])  # Sınıfı indeksle
            boxes.append(bbox)
            classes.append(class_index)

        bboxes.append(boxes)
        labels.append(classes)

    return tf.data.Dataset.from_tensor_slices((images, bboxes, labels))

# Dataseti yükleyelim
dataset = load_data()
print("Dataset başarıyla oluşturuldu!")


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input

# Feature Extractor CNN - Sıfırdan
def create_feature_extractor():
    inputs = Input(shape=(224, 224, 3))

    x = Conv2D(32, (3, 3), activation="relu", padding="same")(inputs)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(256, (3, 3), activation="relu", padding="same")(x)
    x = MaxPooling2D((2, 2))(x)

    model = tf.keras.Model(inputs, x, name="Feature_Extractor")
    return model

feature_extractor = create_feature_extractor()
feature_extractor.summary()


In [4]:
from tensorflow.keras.layers import Conv2D

def create_rpn(feature_map):
    x = Conv2D(256, (3, 3), activation="relu", padding="same")(feature_map)
    objectness_score = Conv2D(1, (1, 1), activation="sigmoid")(x)  # Nesne olup olmadığını tahmin eder
    bbox_regression = Conv2D(4, (1, 1), activation="linear")(x)  # Bounding Box koordinatlarını öğrenir
    return objectness_score, bbox_regression


In [5]:
def create_rcnn_head(feature_map):
    x = Dense(512, activation="relu")(feature_map)
    class_output = Dense(len(class_names), activation="softmax", name="class_output")(x)
    bbox_output = Dense(4, activation="linear", name="bbox_output")(x)
    return class_output, bbox_output


In [ ]:
def create_faster_rcnn():
    inputs = Input(shape=(224, 224, 3))
    
    # 1️⃣ Feature Extraction
    feature_map = create_feature_extractor()(inputs)

    # 2️⃣ Region Proposal Network (RPN)
    objectness_score, bbox_regression = create_rpn(feature_map)

    # 3️⃣ R-CNN Head (Sınıflandırıcı + Bounding Box)
    class_output, final_bbox = create_rcnn_head(feature_map)

    # Modeli birleştirme
    model = tf.keras.Model(inputs=inputs, outputs=[class_output, final_bbox, objectness_score, bbox_regression])
    model.compile(loss=["sparse_categorical_crossentropy", "mse", "binary_crossentropy", "mse"], 
                  optimizer="adam", 
                  metrics=["accuracy"])
    return model

# Modeli oluştur
faster_rcnn = create_faster_rcnn()
faster_rcnn.summary()


In [ ]:
# Modeli eğitme
faster_rcnn.fit(dataset.batch(32), epochs=10, verbose=1)


In [ ]:
import numpy as np

# Test için bir görsel al
test_image, test_bbox, test_label = next(iter(dataset))

# Model ile tahmin yap
pred_class, pred_bbox, obj_score, bbox_reg = faster_rcnn.predict(np.expand_dims(test_image, axis=0))

print("Tahmin edilen sınıf:", class_names[np.argmax(pred_class)])
print("Tahmin edilen bounding box:", pred_bbox)
print("Nesne skoru:", obj_score)
